In [1]:
import spacy
import random

# Creación de datos de entrenamiento:

Fíjate en la estructura de datos de la variable *datos_entrenamiento*: es una **lista de tuplas** cuyo primer elemento es una **oración** y cuyo segundo elemento es un **diccionario de anotaciones**. En esta caso, solo tenemos como clave 'entities', ya que vamos a entrenar un **reconocedor de entidades**. Los valores del diccionario se corresponden con listas de tuplas en las que el primer elemento indica la posición del primer caracter de la entidad, el segundo elemento indica la posición del último caracter de la entidad y el tercer elemento indica la clase de la entidad.

En un entrenamiento real, los **datos de entrenamiento** deberían ser mucho más **numerosos** y los **contextos de aparición** de las entidades deberían ser más **variados**. Sin embargo, pretendemos aquí simplemente estudiar un ejemplo mínimo de entrenamiento.

In [2]:
datos_entrenamiento = [ ('¿Cuánto cuesta un cubata?', {'entities': [(18, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta un vino', {'entities': [(18, 22, 'Bebida')]}),
                        ('¿Cuánto cuesta un rioja?', {'entities': [(18, 23, 'Bebida')]}),
                        ('¿Cuánto cuesta un rueda?', {'entities': [(18, 23, 'Bebida')]}),
                        ('¿Cuánto cuesta un blanco?', {'entities': [(18, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta un rosado?', {'entities': [(18, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta un agua?', {'entities': [(18, 22, 'Bebida')]}),
                        ('¿Cuánto cuesta un tinto?', {'entities': [(18, 23, 'Bebida')]}),
                        ('¿Cuánto cuesta un refresco?', {'entities': [(18, 26, 'Bebida')]}),
                        ('¿Cuánto cuesta un tercio?', {'entities': [(18, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta una caña?', {'entities': [(19, 23, 'Bebida')]}),
                        ('¿Cuánto cuesta una doble?', {'entities': [(19, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta una jarra?', {'entities': [(19, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta una sin?', {'entities': [(19, 22, 'Bebida')]}),
                        ('¿Cuánto cuesta una clara?', {'entities': [(19, 24, 'Bebida')]}),
                        ('¿Cuánto cuesta una sangría?', {'entities': [(19, 26, 'Bebida')]}) ]

In [3]:
from spacy.training.example import Example

# Comenta la siguiente definición de función.

### Debes indicar qué se hace en cada bloque de código. Presta especial atencion a las siguientes funciones:



*   spacy.blank: https://spacy.io/api/top-level#spacy.blank
*   nlp.select_pipes: https://spacy.io/api/language#select_pipes
*   nlp.update: https://spacy.io/api/language/#update

In [4]:
# Crear una función para entrenar un modelo NER
# Requiere dos argumentos:
  # Datos: lista de tuplas
    # Cada tupla consta de un texto y un diccionario de anotaciones
  # Iteraciones: el número de veces que itera para entrenar al modelo

def entrenar(datos,iteraciones):
  # Copia los datos del entrenamiento en una variable loca
    datos_gold = datos

    # crear un modelo spacy en blanco para el lenguaje que seleccionemos
    # en este caso en español ("es")
    # esta es la función gemela de spacy.load()
    nlp = spacy.blank('es')  # Códigos de lenguas: https://spacy.io/usage/models

    # comprueba si no hay un NER existente en el pipe del modelo
    # y si no lo hay, lo crea
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)

    # se añaden las etiquetas de entidades al componente NER del modelo
    for _, anns in datos_gold:
         for ent in anns.get('entities'):
            ner.add_label(ent[2])

    # genera una lista de todos los pipes en el modelo menos "ner"
    # es decir, elimina NER de la lista
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    # ahora desactiva todas las pipes de la lista que ha creado
    # y deja activo solo el pipe de NER
    # esto se hace para asegurarse de que solo se entrene el reconocedor de
    # entidades y no afecte a otras partes del modelo
    with nlp.select_pipes(disable=other_pipes):

      # se inicia el entrenamiento
        optimizer = nlp.begin_training()

        # ciclo de entrenamiento que se repetirá tantas veces como iteraciones hayamos especificado
        for n in range(iteraciones):
            print("Iteración número " + str(n))

            # introducimos un elemento de aleatoriedad
            random.shuffle(datos_gold)

            # creamos un diccionario vacío para hacer un seguimiento de las pérdidas
            losses = {}

            # un bucle que recorre todos los textos y anotaciones en
            # los datos que le hemos dado para el entrenamiento
            for texto, anns in datos_gold:
              doc = nlp.make_doc(texto)
              # crea un ejemplo de entrenamiento a partir del documento y las anotaciones
              ejemplo = Example.from_dict(doc, anns)

              # actualiza el modelo NER con el ejemplo, usando el optimizador y las pérdidas
              nlp.update(
                  [ejemplo],
                  drop=0.2,
                  sgd=optimizer,
                  losses=losses)

              # imprime las pérdidas al acabar cada iteración
            print(losses)

    # devuelve el modelo NER entrenado
    return nlp

In [5]:
ner_bebidas = entrenar(datos_entrenamiento, 40)
ner_bebidas.to_disk("probando")

Iteración número 0
{'ner': 46.594733296427876}
Iteración número 1
{'ner': 6.300818867091676}
Iteración número 2
{'ner': 0.007398072120864127}
Iteración número 3
{'ner': 1.803017773940139e-07}
Iteración número 4
{'ner': 9.25309799089967e-07}
Iteración número 5
{'ner': 6.55526646021362e-08}
Iteración número 6
{'ner': 8.602385429892345e-08}
Iteración número 7
{'ner': 2.8122718750822746e-08}
Iteración número 8
{'ner': 4.357320643582795e-08}
Iteración número 9
{'ner': 1.7350040366419008e-06}
Iteración número 10
{'ner': 1.2314774794700935e-08}
Iteración número 11
{'ner': 4.8531822580905496e-08}
Iteración número 12
{'ner': 8.625657339423589e-09}
Iteración número 13
{'ner': 3.122936138577739e-08}
Iteración número 14
{'ner': 9.328854877938994e-09}
Iteración número 15
{'ner': 1.1887086747391858e-06}
Iteración número 16
{'ner': 5.06816909241418e-09}
Iteración número 17
{'ner': 1.4211795922625136e-09}
Iteración número 18
{'ner': 9.488501254399026e-08}
Iteración número 19
{'ner': 2.4193332409701916

In [6]:
nlp = spacy.load('probando')

In [7]:
from spacy import displacy

In [8]:
doc = nlp(u'¿Cuánto cuesta un cubata? Me querría tomar un cubata. Los cubatas son algo caros. ¿Cuánto cuestan aquí los cubatas? Eres la caña de España. Me querría tomar una caña.')
displacy.render(doc, style='ent', jupyter=True)

# Pregunta de reflexión

Fíjate en lo curioso que es que no aparezca anotado *cubatas* como entidad en la oración *Los cubatas son algo caros*, pero sí en la oración *¿Cuánto cuestan aquí los cubatas?*

En este vídeo uno de los fundadores de SpaCy explica **cómo funciona su modelo reconocimiento de entidades**: https://spacy.io/universe/project/video-spacys-ner-model

Aún habrá muchas cosas de las que cuenta en el vídeo con las que te pierdas, pero... **Serías capaz de intuir la causa del error en este entrenamiento tan escaso**, ¿verdad?

En primer lugar, el error viene de que el modelo de entrenamiento es muy pequeño y además todas sus frases tienen la misma estructura sintáctica. De esta manera, es difícil que el NER pueda reconocer las entidades cuando están en frases con una sintaxis totalmente distinta a la de las frases con las que ha sido entrenado.  